In [1]:
import os, sys
import numpy as np
import scipy.io as scpio
import mne
import pandas as pd
import matplotlib
#matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
from pathlib import Path
from scipy.io import wavfile
import glob
from mne.preprocessing import (ICA, create_eog_epochs, create_ecg_epochs,
                               corrmap)

sys.path.append('../../')
from src import preprocessing as prep
from src import utils

In [2]:
subj_id = 25


In [3]:
utils.annotate_blocks(subjs_dir='../../datasets/data', subj_id=25)

ds directory : /Users/keyvan.mahjoory/k1_analyses/prj_neuroflex/neuroflex_analysis/notebooks/data_preparation/../../datasets/data/subj_25/meg/GMN26_MPIEA0292.NeuroFlex_20220226_04.ds
    res4 data read.
    hc data read.
    Separate EEG position data file not present.
    Quaternion matching (desired vs. transformed):
      -0.46   73.46    0.00 mm <->   -0.46   73.46   -0.00 mm (orig :  -57.62   39.93 -268.68 mm) diff =    0.000 mm
       0.46  -73.46    0.00 mm <->    0.46  -73.46   -0.00 mm (orig :   44.57  -65.59 -265.83 mm) diff =    0.000 mm
     110.23    0.00    0.00 mm <->  110.23    0.00    0.00 mm (orig :   66.58   58.15 -225.23 mm) diff =    0.000 mm
    Coordinate transformations established.
    Polhemus data for 3 HPI coils added
    Device coordinate locations for 3 HPI coils added
    Measurement info composed.
Finding samples for /Users/keyvan.mahjoory/k1_analyses/prj_neuroflex/neuroflex_analysis/notebooks/data_preparation/../../datasets/data/subj_25/meg/GMN26_MPIEA0

In [36]:
subj_name = f'subj_{subj_id}'
stim_channel = 'UPPT001'
subjs_dir = '../../datasets/data/'
meg_dir = os.path.join(subjs_dir, subj_name, 'meg')
behav_dir = os.path.join(subjs_dir, "behav_data", subj_name)
# load blocks info
meg_blocks_info = pd.read_csv(os.path.join(meg_dir, 'nf_blocks_info.csv'), header=0)
n_meg_blocks = meg_blocks_info.shape[0]
# for
block = 2
meg_block_name = meg_blocks_info.origname[block-1]
raw = mne.io.read_raw_ctf(os.path.join(meg_dir, meg_block_name))
events = mne.find_events(raw, stim_channel=stim_channel)

event_initial = np.array([0, 0, 1]).reshape((1, 3))  # to specify the time period before the first trigger!
events_new = np.concatenate((event_initial, events), axis=0)
events_onset_in_sec = events_new[:, 0] / raw.info['sfreq']  # Not included the last event

event_ending = np.array([raw.last_samp, 0, 2]).reshape((1, 3))  # to specify the time period after the last trigger!
events_new_appended = np.concatenate((events_new, event_ending), axis=0)
events_dur_in_sec = (events_new_appended[1:, 0] - events_new_appended[:-1, 0]) / raw.info[
    'sfreq']  # The last events halps find the duration of events
events_correct_response = events[2::4, 2]
annots_correct_response = ['faster' if jev==24 else 'slower' for jev in events_correct_response]

# obtain fm values from MEG data
fms_trials_from_trigger_not_rounded = 8 / events_dur_in_sec[1::4]
fms_trials_from_trigger = np.round(fms_trials_from_trigger_not_rounded * 2)/2  # To round values to 1, 1.5, 2, ...
fms_from_trigger = np.tile(fms_trials_from_trigger.reshape((-1, 1)), (1, 4)).reshape((-1, 1)).ravel()
labels_fm = np.insert(fms_from_trigger, 0, float('nan'))

# obtain fm values from behav data
behav_block_info = pd.read_csv(os.path.join(behav_dir, f'block_{block}.csv'), header=0)
behav_correct_response = behav_block_info.correct_response.to_list()
fms_from_behav = behav_block_info.mod_freq.to_numpy()

assert np.all(fms_trials_from_trigger == fms_from_behav), f'Modulating frequencies do NOT match for block {block}'
assert_correct_response = [annots_correct_response[jev]==behav_correct_response[jev] for jev in range(len(annots_correct_response
                                                                                                            ))]
assert np.all(np.array(assert_correct_response)), f'The event for the correct response does not match  between behavioral and MEG'
# TO DO: Add recording time from Psychtoolbox and MEG and compare ...
#behav_block_info.block_start_datetime[0]
#meg_blocks_info.acqtime[0]

# Annotate using behavioral data
fms = behav_block_info.mod_freq.astype(str).tolist()
phie = behav_block_info.phase_enc.astype(int).astype(str).replace('3', 'pi')
phit = behav_block_info.phase_targ.astype(int).astype(str).replace('3', 'pi')
resp = behav_block_info.eval_response.astype(int).astype(str)
freq_match = behav_block_info.freq_match.astype(str)
events_ = ["/".join([fms[k],f'e{phie[k]}', f't{phit[k]}', f"{freq_match[k][0]}",f"r{resp[k]}"]) for k in range(len(fms))]
event_descs = [f"{j}/{events_[k]}" for k in range(len(events_)) for j in ['e', 'm', 't', 'bad_']]
event_descs.insert(0, 'bad_start')
# Apply annotations to block meg data
block_annots = mne.Annotations(onset=events_onset_in_sec,
                                duration=events_dur_in_sec,
                                description=event_descs,
                                orig_time=raw.info['meas_date'])
raw = raw.set_annotations(block_annots)

# Save the data for block
if write:
    write_path = meg_dir
    write_name = f"block_{block}" + "_raw.fif"  # common meg files should be saved in this format
    raw.save(os.path.join(write_path, write_name), overwrite=True)
# Print list of files in meg directory
    os.system(f"ls -lh {meg_dir}*/")

ds directory : /Users/keyvan.mahjoory/k1_analyses/prj_neuroflex/neuroflex_analysis/notebooks/data_preparation/../../datasets/data/subj_25/meg/GMN26_MPIEA0292.NeuroFlex_20220226_05.ds
    res4 data read.
    hc data read.
    Separate EEG position data file not present.
    Quaternion matching (desired vs. transformed):
      -0.40   73.50    0.00 mm <->   -0.40   73.50   -0.00 mm (orig :  -57.65   39.54 -270.99 mm) diff =    0.000 mm
       0.40  -73.50    0.00 mm <->    0.40  -73.50   -0.00 mm (orig :   44.67  -65.91 -266.32 mm) diff =    0.000 mm
     110.25    0.00    0.00 mm <->  110.25    0.00   -0.00 mm (orig :   66.12   58.28 -226.51 mm) diff =    0.000 mm
    Coordinate transformations established.
    Polhemus data for 3 HPI coils added
    Device coordinate locations for 3 HPI coils added
    Measurement info composed.
Finding samples for /Users/keyvan.mahjoory/k1_analyses/prj_neuroflex/neuroflex_analysis/notebooks/data_preparation/../../datasets/data/subj_25/meg/GMN26_MPIEA0

In [39]:
print(subjs_dir, meg_dir, behav_dir)
meg_blocks_info
events
print(fms_trials_from_trigger)
print(fms_from_behav)

../../datasets/data/ ../../datasets/data/subj_25/meg ../../datasets/data/behav_data/subj_25
[2.5 2.5 2.5 2.5 2.5 2.5 2.5 2.5 2.  2.  2.  2.  2.  2.  2.  2.  3.5 3.5
 3.5 3.5 3.5 3.5 3.5 3.5 4.  4.  4.  4.  4.  4.  4.  4.  3.  3.  3.  3.
 3.  3.  3.  3.  1.  1.  1.  1.  1.  1.  1.  1.  1.5 1.5 1.5 1.5 1.5 1.5
 1.5 1.5]
[2.5 2.5 2.5 2.5 2.5 2.5 2.5 2.5 2.  2.  2.  2.  2.  2.  2.  2.  3.5 3.5
 3.5 3.5 3.5 3.5 3.5 3.5 4.  4.  4.  4.  4.  4.  4.  4.  3.  3.  3.  3.
 3.  3.  3.  3.  1.  1.  1.  1.  1.  1.  1.  1.  1.5 1.5 1.5 1.5 1.5 1.5
 1.5 1.5]


In [40]:

def annotate_blocks(subjs_dir, subj_id, stim_channel = 'UPPT001', write=True):
    """
    This function annotates the MEG data for each block using the behavioral data. 
    It compares the modulating frequencies, correct responses, and the time of recording between the MEG trigger and behavioral data.

    Args:
        subjs_dir (str): path to the directory containing the subjects data. This directory should contain a folder named "behav_data"
          containing the behavioral data for all subjects. And a folder for each subject containing the MEG data.
        subj_id (int): subject id

    Returns:
        None
    """
    subj_name = f'subj_{subj_id}'
    
    meg_dir = os.path.join(subjs_dir, subj_name, 'meg')
    behav_dir = os.path.join(subjs_dir, "behav_data", subj_name)
    # load blocks info
    meg_blocks_info = pd.read_csv(os.path.join(meg_dir, 'nf_blocks_info.csv'), header=0)
    n_meg_blocks = meg_blocks_info.shape[0]
    for block in range(1, n_meg_blocks+1):  # to include the last block as well
        meg_block_name = meg_blocks_info.origname[block-1]
        raw = mne.io.read_raw_ctf(os.path.join(meg_dir, meg_block_name))
        events = mne.find_events(raw, stim_channel=stim_channel)

        event_initial = np.array([0, 0, 1]).reshape((1, 3))  # to specify the time period before the first trigger!
        events_new = np.concatenate((event_initial, events), axis=0)
        events_onset_in_sec = events_new[:, 0] / raw.info['sfreq']  # Not included the last event

        event_ending = np.array([raw.last_samp, 0, 2]).reshape((1, 3))  # to specify the time period after the last trigger!
        events_new_appended = np.concatenate((events_new, event_ending), axis=0)
        events_dur_in_sec = (events_new_appended[1:, 0] - events_new_appended[:-1, 0]) / raw.info[
            'sfreq']  # The last events halps find the duration of events
        events_correct_response = events[2::4, 2]
        annots_correct_response = ['faster' if jev==24 else 'slower' for jev in events_correct_response]

        # obtain fm values from MEG data
        fms_trials_from_trigger_not_rounded = 8 / events_dur_in_sec[1::4]
        fms_trials_from_trigger = np.round(fms_trials_from_trigger_not_rounded * 2)/2  # To round values to 1, 1.5, 2, ...
        fms_from_trigger = np.tile(fms_trials_from_trigger.reshape((-1, 1)), (1, 4)).reshape((-1, 1)).ravel()
        labels_fm = np.insert(fms_from_trigger, 0, float('nan'))

        # obtain fm values from behav data
        behav_block_info = pd.read_csv(os.path.join(behav_dir, f'block_{block}.csv'), header=0)
        behav_correct_response = behav_block_info.correct_response.to_list()
        fms_from_behav = behav_block_info.mod_freq.to_numpy()

        assert np.all(fms_trials_from_trigger == fms_from_behav), f'Modulating frequencies do NOT match for block {block}'
        assert_correct_response = [annots_correct_response[jev]==behav_correct_response[jev] for jev in range(len(annots_correct_response
                                                                                                                  ))]
        assert np.all(np.array(assert_correct_response)), f'The event for the correct response does not match  between behavioral and MEG'
        # TO DO: Add recording time from Psychtoolbox and MEG and compare ...
        #behav_block_info.block_start_datetime[0]
        #meg_blocks_info.acqtime[0]

        # Annotate using behavioral data
        fms = behav_block_info.mod_freq.astype(str).tolist()
        phie = behav_block_info.phase_enc.astype(int).astype(str).replace('3', 'pi')
        phit = behav_block_info.phase_targ.astype(int).astype(str).replace('3', 'pi')
        resp = behav_block_info.eval_response.astype(int).astype(str)
        freq_match = behav_block_info.freq_match.astype(str)
        events_ = ["/".join([fms[k],f'e{phie[k]}', f't{phit[k]}', f"{freq_match[k][0]}",f"r{resp[k]}"]) for k in range(len(fms))]
        event_descs = [f"{j}/{events_[k]}" for k in range(len(events_)) for j in ['e', 'm', 't', 'bad_']]
        event_descs.insert(0, 'bad_start')
        # Apply annotations to block meg data
        block_annots = mne.Annotations(onset=events_onset_in_sec,
                                       duration=events_dur_in_sec,
                                       description=event_descs,
                                       orig_time=raw.info['meas_date'])
        raw = raw.set_annotations(block_annots)

        # Save the data for block
        if write:
            write_path = meg_dir
            write_name = f"block_{block}" + "_raw.fif"  # common meg files should be saved in this format
            raw.save(os.path.join(write_path, write_name), overwrite=True)
    # Print list of files in meg directory
    os.system(f"ls -lh {meg_dir}*/")




In [42]:
annotate_blocks(subjs_dir='../../datasets/data/', subj_id=25, write=False)

ds directory : /Users/keyvan.mahjoory/k1_analyses/prj_neuroflex/neuroflex_analysis/notebooks/data_preparation/../../datasets/data/subj_25/meg/GMN26_MPIEA0292.NeuroFlex_20220226_04.ds
    res4 data read.
    hc data read.
    Separate EEG position data file not present.
    Quaternion matching (desired vs. transformed):
      -0.46   73.46    0.00 mm <->   -0.46   73.46   -0.00 mm (orig :  -57.62   39.93 -268.68 mm) diff =    0.000 mm
       0.46  -73.46    0.00 mm <->    0.46  -73.46   -0.00 mm (orig :   44.57  -65.59 -265.83 mm) diff =    0.000 mm
     110.23    0.00    0.00 mm <->  110.23    0.00    0.00 mm (orig :   66.58   58.15 -225.23 mm) diff =    0.000 mm
    Coordinate transformations established.
    Polhemus data for 3 HPI coils added
    Device coordinate locations for 3 HPI coils added
    Measurement info composed.
Finding samples for /Users/keyvan.mahjoory/k1_analyses/prj_neuroflex/neuroflex_analysis/notebooks/data_preparation/../../datasets/data/subj_25/meg/GMN26_MPIEA0